In [1]:
                        #IMPORT MODULES
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

In [3]:
                        #CONVERT CSV FILES INTO DATAFRAME
# As just done in the data_engineering file, we import the (now cleaned)
# CSV files into dataframes containing metheorogical and logistic data.
metDF = pd.read_csv('clean_measurements.csv')
logDF = pd.read_csv('clean_stations.csv')
metDF.head(10)

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,0.03,73
5,5,USC00519397,2010-01-07,0.06,70
6,6,USC00519397,2010-01-08,0.00,64
7,7,USC00519397,2010-01-09,0.00,68
8,8,USC00519397,2010-01-10,0.00,73
9,9,USC00519397,2010-01-11,0.01,64


In [4]:
logDF

,Unnamed: 0,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [7]:
# Both dataframess have a column "Unnamed: 0" that wouldn't work well later.
# We need to rename it using a compact name (ID).

metDF = metDF.rename(columns={"Unnamed: 0":"ID"})
logDF = logDF.rename(columns={"Unnamed: 0":"ID"})
metDF.head(10)

,ID,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,0.03,73
5,5,USC00519397,2010-01-07,0.06,70
6,6,USC00519397,2010-01-08,0.00,64
7,7,USC00519397,2010-01-09,0.00,68
8,8,USC00519397,2010-01-10,0.00,73
9,9,USC00519397,2010-01-11,0.01,64


In [8]:
logDF

,ID,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [9]:
# Below, the "declarative_base" function, from the sqlalchemy.orm model, is
# used to create the "Base" object.
# This function constructs a base call for declarative class definitions.
# In this case, it's used to model the demographics table as an ORM class.

Base = declarative_base()

In [10]:
# Define the dataframes' ORM (Object Relational Mapper) classes for each
# table.

class Measurements(Base):
    __tablename__ = "metDF"
    ID = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
def __repr__(self):
    return f"id={self.id}, name={self.station}"
    
class Stations(Base):
    __tablename__ = "logDF"
    ID = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Integer)
def __repr__(self):
    return f"id={self.id}, name={self.name}"

In [11]:
# Connect to the database "hawaii".
eng = create_engine("sqlite:///hawaii.sqlite")
conn = eng.connect()

In [12]:
# Use the "create_all" function to create the tables in the database
# correspondent to the two dataframes.
Base.metadata.create_all(eng)

In [13]:
# Create the session, meaning the link from Pandas (Python) to the 
# database.
session = Session(bind=eng)
session.commit()

In [14]:
# Verify that indeed the two dataframe names are in the database.
eng.table_names()

['logDF', 'metDF']

In [15]:
# Below we use Pandas to bulk insert each CSV file into its correspondent 
# database table.

# 1) Populate a database table from a dataframe.

def T1(engine,table,csvfile):

    # Connect to the database.
    conn = engine.connect()

    # Import the CSV file into a dataframe. 
    DF1 = pd.read_csv(csvfile)

    # The orient="records" attributes creates a list of data to write.
    KKK = DF1.to_dict(orient="records")

    # We can optionally delete all rows in the table .
    conn.execute(table.delete())

    # Insert the dataframe KKK into the database in one step ("bulk 
    # insertion").
    conn.execute(table.insert(),KKK)
    
# 2) For each database table, call the function to insert the data.

T1(eng, Stations.__table__, 'clean_stations.csv')
T1(eng, Measurements.__table__, 'clean_measurements.csv')